Pull the nvidia container from build.nvidia.com or ngc.nvidia.com.  If you are using harbor you can use the proxy cache option.

In [16]:
image_name="llama-3.2-11b-vision-instruct"
harbor="vpaif-harbor.set.lab"
harbor_login="admin"
harbor_pwd="Harbor12345"
uri="$harbor/nvidia-cache/nim/meta/llama-3.2-11b-vision-instruct:latest"
NGC_API_KEY="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"
resource_path="/opt/data/resources"

In [3]:
docker pull $uri

latest: Pulling from nvidia-cache/nim/meta/llama-3.2-11b-vision-instruct
Digest: sha256:c1ecd5153ee61cf75498590899d150e599f13e52b24cc6062459f4b4583d6956
Status: Image is up to date for vpaif-harbor.set.lab/nvidia-cache/nim/meta/llama-3.2-11b-vision-instruct:latest
vpaif-harbor.set.lab/nvidia-cache/nim/meta/llama-3.2-11b-vision-instruct:latest


In [4]:
docker run -it --rm --name="$image_name" \
    -e NGC_API_KEY="$NGC_API_KEY" \
    "$uri" \
    bash -c "list-model-profiles"


== NVIDIA Inference Microservice VLM NIM ==

Model: meta/llama-3.2-11b-vision-instruct

Container image Copyright (c) 2016-2024, NVIDIA CORPORATION & AFFILIATES. All rights reserved.

The use of this model is governed by the NVIDIA AI Foundation Models Community License Agreement (found at https://www.nvidia.com/en-us/agreements/enterprise-software/nvidia-ai-foundation-models-community-license-agreement/#:~:text=This%20license%20agreement%20(%E2%80%9CAgreement%E2%80%9D,algorithms%2C%20parameters%2C%20configuration%20files%2C).

A copy of this license can be found under /opt/nim/LICENSE.

SYSTEM INFO
- Free GPUs:
  -  [2235:10de] (0) NVIDIA A40-4C [current utilization: 0%]
MODEL PROFILES
- Compatible with system and runnable:
  - ef4378d956c8f663391615ba64b117e74005bcb39a168b6254b731c0d77c40fa (vllm-bf16-tp1)
  - With LoRA support:
- Incompatible with system:
  - ab89f816413848c86e311123d2ed98af7bcda0c3624b0a6c4d43704b720585d5 (tensorrt_llm-h100-fp8-tp2-latency)
  - a5dcdf7b6790e6084e6

Find the profile id and name in the list that correlates wtih your GPU

In [17]:
profile_id="ef4378d956c8f663391615ba64b117e74005bcb39a168b6254b731c0d77c40fa"
profile_name="vllm-fp16-tp1"
local_model_cache_path="$resource_path/$image_name/$profile_name"_cache
local_model_store_path="$resource_path/$image_name/$profile_name"_model

In [ ]:
docker run -it --rm --name="$image_name" \
    -v "$local_model_cache_path":/opt/nim/.cache \
    -v "$local_model_store_path":/model-repo \
    -e NGC_API_KEY="$NGC_API_KEY" \
    -u "$(id -u root)" \
    "$uri" \
    bash -c "create-model-store --profile $profile_id --model-store /model-repo"


== NVIDIA Inference Microservice VLM NIM ==

Model: meta/llama-3.2-11b-vision-instruct

Container image Copyright (c) 2016-2024, NVIDIA CORPORATION & AFFILIATES. All rights reserved.

The use of this model is governed by the NVIDIA AI Foundation Models Community License Agreement (found at https://www.nvidia.com/en-us/agreements/enterprise-software/nvidia-ai-foundation-models-community-license-agreement/#:~:text=This%20license%20agreement%20(%E2%80%9CAgreement%E2%80%9D,algorithms%2C%20parameters%2C%20configuration%20files%2C).

A copy of this license can be found under /opt/nim/LICENSE.

INFO 2025-03-29 00:33:06.869 pre_download.py:131] Fetching contents for profile ef4378d956c8f663391615ba64b117e74005bcb39a168b6254b731c0d77c40fa
INFO 2025-03-29 00:33:06.872 pre_download.py:138] Copying contents for profile ef4378d956c8f663391615ba64b117e74005bcb39a168b6254b731c0d77c40fa to /model-repo


Log into Harbor to upload model to model store

In [21]:
	docker login "$harbor" -u "$harbor_login" -p "$harbor_pwd"

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/vmware/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


Upload the model to Harbor using PAIS

In [22]:
cd $local_model_store_path
pais models push --modelName $image_name/$profile_name --modelStore $harbor/model-store -t v1

Image Digest: sha256:6e7be131d9f8acadbf3fda926ef08ce483078f938c3d4a25b5fbd67f5647ce45
Tag: v1
To pull this exact model, use
   pais models pull --modelStore vpaif-harbor.set.lab/model-store --modelName llama-3.2-11b-vision-instruct/vllm-fp16-tp1 --digest sha256:6e7be131d9f8acadbf3fda926ef08ce483078f938c3d4a25b5fbd67f5647ce45
